In [134]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib
import itertools
from pylab import rcParams
import pickle
import warnings
warnings.filterwarnings("ignore")

In [18]:
mapping = {}
for i in range(1,7):=
    file_path = 'low_freq/house_' + str(i) + '/labels.dat'
    mapping[i] = {}
    with open(file_path) as file:
        for line in file:
            mapping[i][int(line.split(' ')[0].strip())] = line.split(' ')[1].strip() + '_' + line.split(' ')[0].strip()
            

'bathroom_gfi_7'

In [99]:
df = {}
for i in range(1,7):
    file_path = 'low_freq/house_' + str(i) + '/channel_' + str(i) + '.dat'
    df[i] = pd.read_csv(file_path , sep = ' ', names = ['TimeStamp', mapping[i][1]], dtype = {'TimeStamp': 'int64', mapping[i][1]:'float64'})
    for j in range(2, len(mapping[i])+1):
        curr_file_path =  'low_freq/house_' + str(i) + '/channel_' + str(j) + '.dat'
        curr = pd.read_csv(curr_file_path , sep = ' ', names = ['TimeStamp', mapping[i][j]], dtype = {'TimeStamp': 'int64', mapping[i][j]:'float64'})
        df[i] = pd.merge(df[i], curr, how='inner', on='TimeStamp')
    df[i]['TimeStamp'] = df[i]['TimeStamp'].astype("datetime64[s]")
        

In [104]:
for i in range(1,7):
    for j, row in df[i].iterrows():
        df[i].at[j,'Year'] = row[0].year
        df[i].at[j,'Month'] = row[0].month
        df[i].at[j,'Day'] = row[0].day
        df[i].at[j,'Hour'] = row[0].hour
        df[i].at[j,'Minute'] = row[0].minute
        df[i].at[j,'Seconds'] = row[0].second
        if row[0].weekday() in [5,6] :
            df[i].at[j, 'Holiday'] = 1
        else:
            df[i].at[j, 'Holiday'] = 0
        if row[0].hour in [0,1,2,3,4,5,6,7,20,21,22,23]:
            df[i].at[j, 'Peak Hour'] = 1
        else:
            df[i].at[j, 'Peak Hour'] = 0
        


,TimeStamp,mains_1,mains_2,oven_3,oven_4,refrigerator_5,dishwaser_6,kitchen_outlets_7,kitchen_outlets_8,lighting_9,...,washer_dryer_19,washer_dryer_20,Year,Month,Day,Hour,Minute,Seconds,Holiday,Peak Hour
0,2011-04-18 13:22:13,222.20,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,13.0,0.0,0.0
1,2011-04-18 13:22:16,223.17,119.19,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,16.0,0.0,0.0
2,2011-04-18 13:22:20,223.60,118.92,0.0,0.0,6.0,0.0,34.0,22.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,20.0,0.0,0.0
3,2011-04-18 13:22:23,222.91,119.16,0.0,0.0,6.0,1.0,35.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,23.0,0.0,0.0
4,2011-04-18 13:22:26,222.94,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,26.0,0.0,0.0
5,2011-04-18 13:22:30,222.90,118.76,0.0,0.0,6.0,0.0,33.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,30.0,0.0,0.0
6,2011-04-18 13:22:33,222.96,118.88,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,33.0,0.0,0.0
7,2011-04-18 13:22:37,222.54,118.36,0.0,0.0,6.0,0.0,34.0,21.0,82.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,37.0,0.0,0.0
8,2011-04-18 13:22:40,226.03,119.17,0.0,0.0,6.0,0.0,38.0,21.0,82.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,40.0,0.0,0.0
9,2011-04-18 13:22:44,222.96,119.03,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,0.0,0.0,2011.0,4.0,18.0,13.0,22.0,44.0,0.0,0.0


<h1> Home 1 - Kitchen Outlets </h1>

In [166]:
features = []
for key, value in mapping[1].iteritems():
    if value != 'oven_3':
        features.append(value)
#features.append('Year')
#features.append('Month')
features.append('Day')
features.append('Hour')
features.append('Minute')
features.append('Seconds')
features.append('Holiday')
#features.append('Peak Hour')
train_size = (80 *len(df[1]))/100
x_train = df[1][features][:train_size]
y_train = df[1]['oven_3'][:train_size]
x_test = df[1][features][train_size:]
y_test = df[1]['oven_3'][train_size:]

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)
y_pred_r = regr.predict(x_test)
print("Mean absolute error LR: %.9f"
      % mean_absolute_error(y_test,y_pred_r))

display(y_pred_r)
display(y_test)

Mean absolute error LR: 0.384883621


array([ 0.04525556,  0.04525556,  0.04525556, ...,  0.04525556,
        0.04525556,  0.04525556])

325398    0.0
325399    0.0
325400    0.0
325401    0.0
325402    0.0
325403    0.0
325404    0.0
325405    0.0
325406    0.0
325407    0.0
325408    0.0
325409    0.0
325410    0.0
325411    0.0
325412    0.0
325413    0.0
325414    0.0
325415    0.0
325416    0.0
325417    0.0
325418    0.0
325419    0.0
325420    0.0
325421    0.0
325422    0.0
325423    0.0
325424    0.0
325425    0.0
325426    0.0
325427    0.0
         ... 
406718    0.0
406719    0.0
406720    0.0
406721    0.0
406722    0.0
406723    0.0
406724    0.0
406725    0.0
406726    0.0
406727    0.0
406728    0.0
406729    0.0
406730    0.0
406731    0.0
406732    0.0
406733    0.0
406734    0.0
406735    0.0
406736    0.0
406737    0.0
406738    0.0
406739    0.0
406740    0.0
406741    0.0
406742    0.0
406743    0.0
406744    0.0
406745    0.0
406746    0.0
406747    0.0
Name: oven_3, Length: 81350, dtype: float64